In [10]:
import os
import json
import pandas as pd
import numpy as np
import time
from tqdm import tqdm  # Import tqdm for the progress bar
from evaluator import evaluate
from eslint import EslintProcessor
from google_closure_compiler import GoogleClosureCompiler
from chatgpt_api import ChatGPT  # Import ChatGPT
from gemini_api import gemini_response,gemini_json_response # Import Gemini API
from collections import defaultdict
import enum
from typing_extensions import TypedDict

In [24]:
PROMPT_CODEGEN="""
Task: Generate JavaScript code based on the following description.

Problem Description: {leetcode_desc}

Give your output in this particular schema: {schema}

Requirements:
1. Use modern JavaScript syntax (ES6+)
2. Implement error handling where appropriate
3. Follow best practices for code organization and readability
"""

PROMPT_REFACTOR="""
Task: Refactor the following JavaScript code to improve its quality, readability, and maintainability.

Problem Description: {leetcode_desc}

Original Code: {gt_code}

Give your output in this particular schema: {schema}

Requirements:
1. Improve the run-time of the algorithm
2. Improve the readability and quality of algorithm
"""

PROMPT_REFACTOR_V2="""
Task: Refactor the following JavaScript code to improve its quality, readability, and maintainability.

Original Code: {gt_code}

Give your output in this particular schema: {schema}

Requirements:
1. Improve the run-time of the algorithm
2. Improve the readability and quality of algorithm
"""


# Experiment 3: Refactor with metadata from LeetCode added
PROMPT_REFACTOR_V3="""
Task: Refactor the following JavaScript code to improve its quality, readability, and maintainability.

Problem Description: {leetcode_desc}

Examples: {examples}

Constraint: {constraint}

Original Code: {gt_code}

Give your output in this particular schema: {schema}

Requirements:
1. Improve the run-time of the algorithm
2. Improve the readability and quality of algorithm
"""

PROMPT_REFACTOR_V4="""
Task: Refactor the following JavaScript code to improve its quality, readability, and maintainability.

Problem Description: {leetcode_desc}

Original Code: {gt_code}

Give your output in this particular schema: {schema}

Requirements:
1. Improve the run-time of the algorithm
2. Improve the readability and quality of algorithm
3. Use the suggestions provided the Linter to optimize the code

Linter output of the original code: {linter_output}
"""

# Experiment 5: Refactor with more specific requirements (e.g., make the solution 10 lines or less) (not trivial, the above cases first)
PROMPT_REFACTOR_V5="""
Task: Refactor the following JavaScript code to improve its quality, readability, and maintainability.

Problem Description: {leetcode_desc}

Original Code: {gt_code}

Examples: {examples}

Constraint: {constraint}

Give your output in this particular schema: {schema}

Requirements:
1. Improve the run-time of the algorithm if possible
2. Improve the readability and quality of algorithm
3. Algorithm Optimization: Suggest a more efficient algorithm in terms of run-time if possible. Consider using appropriate data structures like hash tables for O(1) lookups or binary search trees for O(log n) operations 
4. Code Optimization: Minimize unnecessary loops and nested loops. Use efficient built-in functions and methods specific to the programming language. Reduce redundant calculations by caching results or using dynamic programming
5. Memory Optimization: Optimize memory usage by using appropriate data types. Consider in-place algorithms to reduce space complexity.
"""

out_schema = {
    "code": "generated javascript code",
    "rationale": "explain your approach keep it under 200 words"
}
gemini_schema = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
        "properties": {
            "code": {"type": "string"},
            "rationale": {"type": "string"}
        },
        "required": ["code", "rationale"]
    }
}



In [2]:
import pandas as pd

df = pd.read_csv('Code_Generation_Dataset_with_Linter_Outputs.csv')

print(df.head())
print(df.columns)

   Problem Number                                        Name  \
0               1                 Create Hello World Function   
1               2                                   Counter 2   
2               3  Apply Transform Over Each Element in Array   
3               4                  Filter Elements from Array   
4               5                 Array Reduce Transformation   

                                         Description  \
0  Write a function createHelloWorld. It should r...   
1  Write a function createCounter. It should acce...   
2  Given an integer array arr and a mapping funct...   
3  Given an integer array arr and a filtering fun...   
4  Given an integer array nums, a reducer functio...   

                                            Examples  \
0  Example 1:\n\nInput: args = []\nOutput: "Hello...   
1  Example 1:\n\nInput: init = 5, calls = ["incre...   
2  Example 1:\n\nInput: arr = [1,2,3], fn = funct...   
3  Example 1:\n\nInput: arr = [0,10,20,30], fn =

In [22]:
# Prompts with their explicit formatting
prompts = {
    "CODEGEN": lambda row: PROMPT_CODEGEN.format(
        leetcode_desc=row['Description'],
        schema=row['Function Header']
    ),
    "REFACTOR": lambda row: PROMPT_REFACTOR.format(
        leetcode_desc=row['Description'],
        gt_code=row['Solution'],
        schema=row['Function Header']
    ),
    "REFACTOR_V2": lambda row: PROMPT_REFACTOR_V2.format(
        gt_code=row['Solution'],
        schema=row['Function Header']
    ),
    "REFACTOR_V3": lambda row: PROMPT_REFACTOR_V3.format(
        leetcode_desc=row['Description'],
        examples=row['Examples'],
        constraint=row['Constraints'],
        gt_code=row['Solution'],
        schema=row['Function Header']
    ),
    "REFACTOR_V4": lambda row: PROMPT_REFACTOR_V4.format(
        leetcode_desc=row['Description'],
        gt_code=row['Solution'],
        schema=row['Function Header'],
        linter_output=row['Linter Outputs']
    ),
    "REFACTOR_V5": lambda row: PROMPT_REFACTOR_V5.format(
        leetcode_desc=row['Description'],
        gt_code=row['Solution'],
        examples=row['Examples'],
        constraint=row['Constraints'],
        schema=row['Function Header']
    )
}

In [25]:
models_with_delays = {
    "gemini-1.5-flash": 1,
    "gemini-1.5-pro": 1
}

In [26]:
import pandas as pd
import json
import time
import os
from tqdm import tqdm

# Max tokens for the response
MAX_TOKENS_ANSWER = 2000

# Ensure output directory exists
output_dir = "results"
os.makedirs(output_dir, exist_ok=True)

# Maximum number of retries
MAX_RETRIES = 5

# Wrap outer loop for models in tqdm
for model_name, delay in tqdm(models_with_delays.items(), desc="Models"):
    for prompt_name, prompt_formatter in tqdm(prompts.items(), desc=f"Prompts for {model_name}", leave=False):
        # Initialize lists to store results for this combination
        gt_code_lst = []
        lvl_lst = []
        ans = []
        rat = []

        # Wrap DataFrame iteration in tqdm for row-level progress
        for index, row in tqdm(df.iterrows(), desc=f"Examples for {prompt_name}", total=len(df), leave=False):
            retry_count = 0
            while retry_count < MAX_RETRIES:
                try:
                    # Generate the prompt using the specific formatter
                    filled_prompt = (
                        prompt_formatter(row)
                        + "\n"
                        + "Provide output in valid JSON. The data schema should be like this: "
                        + json.dumps(out_schema)
                    )

                    # Send the prompt to Gemini API
                    response_txt = gemini_json_response(
                        filled_prompt, gemini_schema, model_name, MAX_TOKENS_ANSWER
                    )
                    
                    # Parse the response
                    response_json = json.loads(response_txt[1:-1])

                    print("response_json: ")
                    print(response_json)

                    # Append results
                    gt_code_lst.append(row["Solution"])  # Ground truth code
                    lvl_lst.append(row["Difficulty"])  # Difficulty level
                    ans.append(response_json["code"])  # Refactored code
                    rat.append(response_json["rationale"])  # Rationale

                    # Break out of retry loop on success
                    break

                except Exception as e:
                    retry_count += 1
                    print(
                        f"Error with model {model_name}, prompt {prompt_name}, example {index} (Attempt {retry_count}): {str(e)}"
                    )
                    if retry_count >= MAX_RETRIES:
                        # Log error after exceeding retries
                        gt_code_lst.append(row["Solution"])  # Ground truth code
                        lvl_lst.append(row["Difficulty"])  # Difficulty level
                        ans.append("Error")
                        rat.append(str(e))
                        print(
                            f"Skipping example {index} after {MAX_RETRIES} attempts."
                        )

                # Delay before the next retry to prevent spamming the API
                time.sleep(delay)

            # Delay to prevent API rate limiting
            time.sleep(delay)

        # Create a DataFrame for the results
        output_df = pd.DataFrame()
        output_df["gt_code"] = gt_code_lst
        output_df["level"] = lvl_lst
        output_df["refactor_code"] = ans
        output_df["rationale"] = rat

        # Save results to a separate CSV file for each prompt-model combination
        output_file = os.path.join(
            output_dir, f"{model_name}_{prompt_name}_results.csv"
        )
        output_df.to_csv(output_file, index=False)
        print(f"Results saved to '{output_file}'")


Models:   0%|          | 0/1 [00:00<?, ?it/s]


response_json: 
{'code': '/**\n * @return {Function}\n */\nvar createHelloWorld = function() {\n    return () => "Hello World";\n};\n\n/**\n * const f = createHelloWorld();\n * f(); // "Hello World"\n */', 'rationale': 'The original code was already quite efficient and readable. This refactoring makes it slightly more concise by using an arrow function expression `() => "Hello World"` which has an implicit return.  This maintains the core functionality of returning a function that always returns "Hello World" while slightly improving the code\'s brevity. The use of arrow functions also lexically binds the `this` keyword, which can be beneficial in certain situations, though not directly relevant in this specific example.  No change in run-time is expected, as the underlying operation remains the same.'}


response_json: 
{'code': '/**\n * @param {integer} init\n * @return { increment: Function, decrement: Function, reset: Function }\n */\nvar createCounter = function(init) {\n    let currentCount = init;\n\n    const increment = () => ++currentCount;\n    const decrement = () => --currentCount;\n    const reset = () => (currentCount = init);\n\n    return { increment, decrement, reset };\n};', 'rationale': "The provided code is already quite efficient and readable. This refactoring makes a few minor improvements:\n\n1. Renames `presentCount` to `currentCount` for clarity.\n2. Uses arrow functions for `increment`, `decrement`, and `reset` to make the code more concise while maintaining readability.\n3. Uses const for the inner functions as they won't be reassigned.\nThese changes improve the overall readability and conciseness of the code without affecting its performance."}


response_json: 
{'code': '/**\n * @param {number[]} arr\n * @param {Function} fn\n * @return {number[]}\n */\nvar map = function(arr, fn) {\n    const transformedArr = [];\n    for (let i = 0; i < arr.length; i++) {\n        transformedArr[i] = fn(arr[i], i);\n    }\n    return transformedArr;\n};', 'rationale': 'This refactoring replaces the forEach loop with a standard for loop.  Generally, for loops offer better performance than forEach, especially for large arrays, as they avoid the overhead of function calls for each element.  This change improves run-time performance without sacrificing readability. The code also maintains the original functionality and intent, ensuring that the returned array contains the transformed elements based on the provided mapping function.'}


response_json: 
{'code': '/**\n * @param {number[]} arr\n * @param {Function} fn\n * @return {number[]}\n */\nvar filter = function(arr, fn) {\n    const filteredArr = [];\n    for (let i = 0; i < arr.length; i++) {\n        if (fn(arr[i], i)) {\n            filteredArr.push(arr[i]);\n        }\n    }\n    return filteredArr;\n};', 'rationale': 'This refactoring focuses on enhancing readability and aligning with the provided schema requirements. The core logic remains efficient, iterating through the array once. Key changes include:\n\n1. Renaming `result` to `filteredArr` for clarity and consistency with the problem description.\n2. Maintaining the existing loop structure, which offers optimal performance for this filtering task.\n3. Adhering strictly to the provided schema for output, ensuring the code integrates seamlessly into the expected environment.\n\nThis approach prioritizes code clarity and adherence to specifications while preserving the efficient linear time complexity of 

...

response_json: 
{'code': "/**\n * @param {string} s\n * @param {string} p\n * @return {boolean}\n */\nvar isMatch = function(s, p) {\n    const m = s.length;\n    const n = p.length;\n\n    const dp = Array(m + 1).fill(false).map(() => Array(n + 1).fill(false));\n    dp[0][0] = true; // Empty string matches empty pattern\n\n    // Initialize first row (empty string)\n    for (let j = 1; j <= n; j++) {\n        if (p[j - 1] === '*') {\n            dp[0][j] = dp[0][j - 2];\n        }\n    }\n\n    for (let i = 1; i <= m; i++) {\n        for (let j = 1; j <= n; j++) {\n            if (p[j - 1] === '.' || p[j - 1] === s[i - 1]) {\n                dp[i][j] = dp[i - 1][j - 1];\n            } else if (p[j - 1] === '*') {\n                dp[i][j] = dp[i][j - 2];\n                if (p[j - 2] === '.' || p[j - 2] === s[i - 1]) {\n                    dp[i][j] = dp[i][j] || dp[i - 1][j];\n                }\n            }\n        }\n    }\n\n    return dp[m][n];\n};", 'rationale': 'This refactore

response_json: 
{'code': '/**\n * @param {number[]} nums\n * @return {number}\n */\nvar maxCoins = function(nums) {\n    const n = nums.length;\n    nums.push(1);\n    nums.unshift(1);\n\n    const dp = Array(n + 2).fill(0).map(() => Array(n + 2).fill(0));\n\n    for (let len = 1; len <= n; len++) {\n        for (let i = 1; i <= n - len + 1; i++) {\n            const j = i + len - 1;\n            for (let k = i; k <= j; k++) {\n                dp[i][j] = Math.max(dp[i][j], nums[i - 1] * nums[k] * nums[j + 1] + dp[i][k - 1] + dp[k + 1][j]);\n            }\n        }\n    }\n\n    return dp[1][n];\n};', 'rationale': "The provided code utilizes dynamic programming to determine the maximum coins obtainable by bursting balloons. To address boundary conditions, the 'nums' array is padded with 1s at both ends. A 2D DP table, 'dp,' is employed, where dp[i][j] stores the maximum coins collected by bursting balloons within the range [i, j]. The code iterates through different subarray lengths an

response_json: 
{'code': "/**\n * @param {character[][]} board\n * @return {void} Do not return anything, modify board in-place instead.\n */\nvar solveSudoku = function(board) {\n    const n = 9;\n    const isValid = (row, col, num) => {\n        for (let i = 0; i < n; i++) {\n            if (board[row][i] === num || board[i][col] === num) return false;\n        }\n        const startRow = 3 * Math.floor(row / 3);\n        const startCol = 3 * Math.floor(col / 3);\n        for (let i = 0; i < 3; i++) {\n            for (let j = 0; j < 3; j++) {\n                if (board[startRow + i][startCol + j] === num) return false;\n            }\n        }\n        return true;\n    };\n\n    const solve = () => {\n        for (let row = 0; row < n; row++) {\n            for (let col = 0; col < n; col++) {\n                if (board[row][col] === '.') {\n                    for (let num = 1; num <= 9; num++) {\n                        const numStr = num.toString();\n                        if (



Models: 100%|██████████| 1/1 [21:45<00:00, 1305.15s/it]

Results saved to 'results/gemini-1.5-pro_REFACTOR_V3_results.csv'
